In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import albumentations as A
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from pathlib import Path
import shutil

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [2]:
import numpy as np
import os
import cv2
import albumentations as A
import matplotlib.pyplot as plt

# Define the path to the training images
train_images_path = '/kaggle/input/ua-detrac-dataset/content/UA-DETRAC/DETRAC_Upload/images/train'

max_batches = 200
num_batch_count= 0

# Generator to load images in batches
def load_images_in_batches(folder, batch_size=50):
    images = []
    filenames = []
    batch_count = 0
    global num_batch_count
    print(num_batch_count)
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):  # Assuming images are in JPG format
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
                filenames.append(Path(filename).name)
                batch_count += 1
                
        # Yield the batch if the count hits batch_size
        if batch_count == batch_size:
            yield images, filenames
            images = []  # Reset for next batch
            filenames = []  # Reset for next batch
            batch_count = 0
            num_batch_count += 1  # Increment batch counter
            
        # Stop yielding if max_batches is reached
        if num_batch_count >= max_batches:
            break
    # Yield the remaining images if any
    if images:
        yield images, filenames

batch_size = 5
# Load a small batch of images for demonstration (e.g., first batch)
image_batches = load_images_in_batches(train_images_path, batch_size=batch_size)
# first_batch = next(image_batches)  # Get the first batch
# for _ in image_batches:A
#     batch = next(image_batch)    

# Define augmentation functions
def add_fog(image):
    aug = A.Compose([A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, alpha_coef=0.1, p=1.0)])
    augmented = aug(image=image)
    return augmented['image']

def add_rain(image):
    aug = A.Compose([A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, blur_value=3, brightness_coefficient=0.8, rain_type='heavy', p=1.0)])
    augmented = aug(image=image)
    return augmented['image']

def add_snow(image):
    aug = A.Compose([A.RandomSnow(snow_point_lower=0.3, snow_point_upper=0.5, brightness_coeff=2.5, p=1.0)])
    augmented = aug(image=image)
    return augmented['image']

def adjust_brightness_contrast(image):
    aug = A.Compose([A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1.0)])
    augmented = aug(image=image)
    return augmented['image']

# Define output folder for saving augmented images
output_folder = '/kaggle/working'
os.makedirs(output_folder, exist_ok=True)

weather = ['fog','rain','snow']
for weather_singular in weather: # weather_singular is bad naming
    os.system(f"mkdir {os.path.join(output_folder, 'images_'+ weather_singular)}")
    os.system(f"mkdir {os.path.join(output_folder, 'labels_'+ weather_singular)}")
os.listdir(output_folder)

        
k = 0
bacth_generator = load_images_in_batches(train_images_path, batch_size)
for (batch,filenames) in bacth_generator:
    # batch = next(image_batches) 
    # Applying augmentations to the first batch of images and displaying them
    for i, (image, filename) in enumerate(zip(batch,filenames)):
        fog_image = add_fog(image)
        rain_image = add_rain(image)
        snow_image = add_snow(image)
        bright_contrast_image = adjust_brightness_contrast(image)

        # Save the augmented images to the output folder
        augmented_images = [add_fog(image), add_rain(image), add_snow(image)]
        input_label_path = Path(os.path.join('/kaggle/input/ua-detrac-dataset/content/UA-DETRAC/DETRAC_Upload/labels/train',Path(filename).stem+".txt"))
        if not input_label_path.exists():
            continue
        for j, aug_image in enumerate(augmented_images):
            output_path = os.path.join(output_folder, 'images_'+ weather[j], filename)
            cv2.imwrite(output_path, aug_image)
            #             print(output_path)
            
            output_label_path = os.path.join(output_folder, 'labels_'+ weather[j], Path(filename).stem+".txt")
            shutil.copy(input_label_path, output_label_path)
            #             print(output_label_path)
    print(f"Augmented images saved in {output_folder} {k}-th batch")
    k+=1

0
Augmented images saved in /kaggle/working 0-th batch
Augmented images saved in /kaggle/working 1-th batch
Augmented images saved in /kaggle/working 2-th batch
Augmented images saved in /kaggle/working 3-th batch
Augmented images saved in /kaggle/working 4-th batch
Augmented images saved in /kaggle/working 5-th batch
Augmented images saved in /kaggle/working 6-th batch
Augmented images saved in /kaggle/working 7-th batch
Augmented images saved in /kaggle/working 8-th batch
Augmented images saved in /kaggle/working 9-th batch
Augmented images saved in /kaggle/working 10-th batch
Augmented images saved in /kaggle/working 11-th batch
Augmented images saved in /kaggle/working 12-th batch
Augmented images saved in /kaggle/working 13-th batch
Augmented images saved in /kaggle/working 14-th batch
Augmented images saved in /kaggle/working 15-th batch
Augmented images saved in /kaggle/working 16-th batch
Augmented images saved in /kaggle/working 17-th batch
Augmented images saved in /kaggle/wo

Trying YOLO v5 with speed detection


In [3]:
# import torch
# import cv2
# import numpy as np

# # Load the YOLOv5 model correctly from the ultralytics repository
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# # Path to the folder containing the frames
# image_folder = '/kaggle/input/ua-detrac-dataset/content/UA-DETRAC/DETRAC_Upload/images/train'

# # Frame rate of the video (e.g., 30 FPS)
# frame_rate = 30

# # Conversion factor from pixels to real-world speed (you need camera calibration for this)
# pixel_to_speed_conversion = 0.05  # Example value, needs to be calibrated

# # Function to calculate speed
# def calculate_speed(displacement, time_interval):
#     return displacement * pixel_to_speed_conversion / time_interval

# # Load frames
# def load_frames(image_folder):
#     images = []
#     for filename in sorted(os.listdir(image_folder)):
#         if filename.endswith(".jpg"):  
#             img = cv2.imread(os.path.join(image_folder, filename))
#             images.append(img)
#     return images

# # Track vehicles and calculate speed
# def track_and_calculate_speed(frames, model, frame_rate):
#     speeds = []
#     prev_frame_boxes = None

#     for i in range(1, len(frames)):
#         # Detect vehicles in the current frame
#         results = model(frames[i])
#         current_frame_boxes = results.xyxy[0].cpu().numpy()  # Bounding box coordinates (x1, y1, x2, y2)

#         # If this is not the first frame, track and calculate speed
#         if prev_frame_boxes is not None:
#             # For simplicity, let's assume we are matching vehicles by the closest bounding boxes
#             for j, current_box in enumerate(current_frame_boxes):
#                 # Find the closest bounding box from the previous frame
#                 prev_box = min(prev_frame_boxes, key=lambda box: np.linalg.norm(current_box[:2] - box[:2]))

#                 # Calculate the displacement (center points of the bounding boxes)
#                 current_center = np.array([(current_box[0] + current_box[2]) / 2, (current_box[1] + current_box[3]) / 2])
#                 prev_center = np.array([(prev_box[0] + prev_box[2]) / 2, (prev_box[1] + prev_box[3]) / 2])
#                 displacement = np.linalg.norm(current_center - prev_center)

#                 # Calculate the speed (displacement / time interval)
#                 time_interval = 1 / frame_rate
#                 speed = calculate_speed(displacement, time_interval)
#                 speeds.append((i, speed))  # Store the frame index and speed for annotation

#         # Update previous frame boxes for the next iteration
#         prev_frame_boxes = current_frame_boxes

#     return speeds

# # Load the frames
# frames = load_frames(image_folder)

# # Track and calculate speeds
# speeds = track_and_calculate_speed(frames, model, frame_rate)

# # Display calculated speeds
# for frame_index, speed in speeds:
#     print(f"Frame {frame_index}: Speed = {speed:.2f} units")


In [4]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

# # Define custom dataset class
# class VehicleSpeedDataset(Dataset):
#     def __init__(self, image_folder, annotations):
#         self.images = sorted(os.listdir(image_folder))
#         self.annotations = annotations  # Assuming annotations is a list of (bounding_box, speed) tuples

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         img_path = os.path.join(image_folder, self.images[idx])
#         image = cv2.imread(img_path)
#         bbox, speed = self.annotations[idx]
#         return image, bbox, speed

# # Define model architecture
# class SpeedDetectionModel(nn.Module):
#     def __init__(self, pretrained_model):
#         super(SpeedDetectionModel, self).__init__()
#         self.base_model = pretrained_model  # Pre-trained YOLOv5 or other detection model
#         self.fc = nn.Linear(1024, 1)  # Add a fully connected layer for speed prediction

#     def forward(self, x):
#         features = self.base_model(x)
#         speed = self.fc(features)  # Predict speed
#         return speed

# # Load pre-trained YOLO model
# yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# # Create your speed detection model
# model = SpeedDetectionModel(yolo_model)

# # Define loss function and optimizer
# criterion = nn.MSELoss()  # Mean Squared Error for regression
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Train the model
# def train_model(model, train_loader, num_epochs=10):
#     model.train()
#     for epoch in range(num_epochs):
#         running_loss = 0.0
#         for images, bboxes, speeds in train_loader:
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, speeds)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()

#         print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# # Prepare dataset and dataloader
# annotations = [(bbox, speed) for bbox, speed in zip(bboxes_list, speed_list)]
# train_dataset = VehicleSpeedDataset(image_folder, annotations)
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# # Train the model
# train_model(model, train_loader)
